# Configurações

## Importações

In [133]:
import os
import sys
sys.path.append('../functions')


import pandas as pd
import numpy as np

from preprocessing import apply_boxcox_transform
from sklearn.preprocessing import MinMaxScaler, StandardScaler, FunctionTransformer
from scipy.stats import boxcox
from sklearn.pipeline import Pipeline
import joblib

## Definição do Dataframe

In [134]:
df = pd.read_csv('../datasets/data_clean.csv')

# Pré-processamento

## Seleção de Features

In [135]:
df_selecao_6f = df.copy()
df_selecao_4f = df.copy()

In [136]:
df_selecao_6f.columns

Index(['id', 'name', 'artists', 'duration_ms', 'release_date', 'year',
       'acousticness', 'danceability', 'energy', 'instrumentalness',
       'liveness', 'loudness', 'speechiness', 'tempo', 'valence', 'mode',
       'key', 'popularity', 'explicit'],
      dtype='object')

In [137]:
df_selecao_6f.drop(columns=['name', 'release_date', 'artists', 'duration_ms', 'key', 'mode', 'tempo', 'loudness', 'valence', 'liveness', 'explicit', 'year'], inplace=True)

df_selecao_4f.drop(columns=['name', 'release_date', 'artists', 'duration_ms', 'key', 'mode', 'tempo', 'loudness', 'valence', 'liveness', 'explicit', 'year', 'speechiness', 'instrumentalness'], inplace=True)

In [138]:
df_selecao_6f.columns

Index(['id', 'acousticness', 'danceability', 'energy', 'instrumentalness',
       'speechiness', 'popularity'],
      dtype='object')

In [139]:
df_selecao_4f.columns

Index(['id', 'acousticness', 'danceability', 'energy', 'popularity'], dtype='object')

In [140]:
df_features_6f = df_selecao_6f.copy()
df_features_6f.drop(columns=['id'], inplace=True)

df_features_4f = df_selecao_4f.copy()
df_features_4f.drop(columns=['id'], inplace=True)

A feature "genre" possui os gêneros das músicas, sendo uma string, com todos os gêneros da música, separados por vírgula. 

O objetivo desse pré-processamento é dividir essa string e aplicar uma técnica para criar colunas binárias dos gêneros.

## Normalização

### Pipelines de treinamento

#### 6 Features

In [141]:
pipeline_6f = Pipeline([
    ('boxcox_transform', FunctionTransformer(apply_boxcox_transform, validate=False)),
    ('popularity_scaler', MinMaxScaler()),
    ('standard_scaler', StandardScaler())
])

pipeline_pred_6f = Pipeline([
    ('popularity_scaler', MinMaxScaler()),
    ('standard_scaler', StandardScaler())
])

numeric_cols_6f = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'speechiness', 'popularity']

pipeline_pred_6f.fit(df_selecao_6f[numeric_cols_6f])

df_selecao_6f[numeric_cols_6f] = pipeline_6f.fit_transform(df_selecao_6f[numeric_cols_6f])

#### 4 Features

In [142]:
pipeline_4f = Pipeline([
    ('popularity_scaler', MinMaxScaler()),
    ('standard_scaler', StandardScaler())
])

numeric_cols_4f = ['acousticness', 'danceability', 'energy', 'popularity']

df_selecao_4f[numeric_cols_4f] = pipeline_4f.fit_transform(df_selecao_4f[numeric_cols_4f])

In [143]:
print("✅ Pipelines de normalização aplicadas com sucesso!")
print(f"\n📊 df_selecao_6f - Primeiras linhas após normalização (COM ID):")
print(df_selecao_6f[['id'] + numeric_cols_6f].head())
print(f"\n📊 df_features_6f - Primeiras linhas após normalização (SEM ID):")
print(df_features_6f[numeric_cols_6f].head())

✅ Pipelines de normalização aplicadas com sucesso!

📊 df_selecao_6f - Primeiras linhas após normalização (COM ID):
                       id  acousticness  danceability    energy  \
0  0gNNToCW3qjabgTyBSjt3H      0.084389      0.352754  0.192262   
1  0tMgFpOrXZR6irEOLNWwJL     -0.704189     -0.970359 -0.032130   
2  2ZywW3VyVx6rrlrX75n3JB     -1.044579     -1.517854  0.730801   
3  6DdWA7D1o5TU2kXWyCLcch     -0.818360     -0.953250  0.210961   
4  3vMmwsAiLDCfyc1jl76lQE     -0.818360     -0.953250  0.210961   

   instrumentalness  speechiness  popularity  
0          0.406229    -0.727591   -0.257460  
1          1.427540    -0.823343   -0.489128  
2          0.798087    -0.149776    0.437542  
3          1.405054    -0.818825    0.252208  
4          1.405054    -0.818825    0.159541  

📊 df_features_6f - Primeiras linhas após normalização (SEM ID):
   acousticness  danceability  energy  instrumentalness  speechiness  \
0        0.5250         0.600   0.540           0.00305       0

# Gerar o dataframe

In [144]:
df_selecao_6f.to_csv('../datasets/pre_processing_6f.csv', index=False, encoding='utf-8')

df_selecao_4f.to_csv('../datasets/pre_processing_4f.csv', index=False, encoding='utf-8')

In [145]:
df_features_6f.to_csv('../datasets/features_6f.csv', index=False, encoding='utf-8')

df_features_4f.to_csv('../datasets/features_4f.csv', index=False, encoding='utf-8')

# Gerar as pipelines

In [146]:

os.makedirs('../models', exist_ok=True)

joblib.dump(pipeline_6f, '../models/pipeline_preprocessor_6f.joblib')
joblib.dump(pipeline_pred_6f, '../models/pipeline_predictor_6f.joblib')
joblib.dump(pipeline_4f, '../models/pipeline_preprocessor_4f.joblib')

['../models/pipeline_preprocessor_4f.joblib']

# Resultados

Foram removidas 12 features e mantido apenas 6.